In [ ]:
# This iPython notebook implements a probabilistic formulation of the ranking principal curve model described
# in the Chun-Guo Li et al. paper "Unsupervised Ranking of Multi-Attribute Objects Based on Principal Curves"
# (https://arxiv.org/abs/1402.4542). The model learns to rank 171 countries by four "quality of life" (QoL)
# indicators drawn from GAPMINDER data (https://www.gapminder.org/data/), following the Zinovyev and Gorban
# paper "Nonlinear Quality of Life Index" (https://arxiv.org/abs/1008.4063). We compare our inference results
# to the ordering of countries presented in Figure 1 of that paper (i.e we perform eyeball validation). More
# rigorous validation and model comparison methods are planned for future work. We implement our model using
# the probabilistic programming language PyMC (https://www.pymc.io).
#
# The procedure given in the Li et al. paper uses mathematical optimization to minimize the error between the
# reconstruction of feature vectors given by a ranking principal curve and the true/observed feature values.
# In this work, we use Bayesian inferece/inverse probability to maximize the probability that the observed
# feature values are draws from the posterior predictive distribution over feature vectors derived from a 
# posterior distirbution over raking principal curves.

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import pytensor.tensor as pt

rng = np.random.default_rng(0)

In [ ]:
# Here, we prepare the functions used for feature endineering. 

# We use Beta-distributed random variables to model the response/data distribution. Because the support in 
# PyMC is bounded by (0, 1), when rescaling the features in our data set we set the bounds to be slightly 
# higher than zero and slightly lower than one.
near_zero = 1e-12

def min_max_scale(x, min=near_zero, max=1. - near_zero):
    x_scaled = (x - np.min(x, axis=0)) / (np.max(x, axis=0) - np.min(x, axis=0))
    x_scaled = x_scaled * (max - min) + min
    return x_scaled

# some features (like life expectancy) increase in value as the QoL associated with that feature increases.
# Others (like infant morality rate) decrease in value as the associated QoL increases. We reverse the scale
# for features of the latter kind to match features of the former kind.
def range_reverse(x):
    x_reversed = 1. - x
    return x_reversed

In [ ]:
# Read in the data set prepared from the GAPMINDER website.
gapminder_series = pd.read_csv("./data/gapminder_quality_of_life_2005_dataset.csv", index_col="country")
gapminder_data = gapminder_series.values

gapminder_data = min_max_scale(gapminder_data)

gapminder_data[:, 1] = range_reverse(gapminder_data[:, 1])
gapminder_data[:, 2] = range_reverse(gapminder_data[:, 2])

# As in the Li et al. paper, we transpose the data matrix to comply with the matrix form of the likelihood function
X = np.transpose(gapminder_data)

In [ ]:
# Here, we define the model prior probability distribution parameters
dims, num_scores = X.shape

# Initialize the Bezier control points to form a moderately sigmoidal curve shape
alpha_prior = np.repeat([.75], dims) + .25 * (rng.random(dims) - .5)
p_1_prior = np.repeat([.3125], dims) + .25 * (rng.random(dims) - .5)
p_2_prior = np.repeat([.6875], dims) + .25 * (rng.random(dims) - .5)

# Use the means across features of a country as priors for their scalar score values.
s_prior = np.mean(X, axis=0)

# In the mean/variance parameterization od Beta distrubutions, sigmas are tiny. These values
# were chosen through trial and error such that the MCMC sampler would initialize at a point in the
# search space that yields a valid initial logprob. We also bound the sigmas for the same reason.
#
# Separately, all points share a common sigma, and all scores share a common sigma as a form of 
# regularization. Each dimension of the response variable is modeled separately with separate sigmas.
# We learn the values of response sigmas because calculating them directly from the data would assume
# (incorrectly) that the data are Gaussian-distributed. (*Confusing phrasing?)
point_sigma_prior = .1
score_sigma_prior = .135
X_reconstruction_sigma_prior = np.repeat(.125, dims)

sigma_prior_sigma = .1
sigma_prior_upper_bound_delta = .01

In [ ]:
# Here, we define our probabilistic model. Our sigmas are normally-distributed, but bounded as described
# earlier. As described in the Li et al. paper, to constrain the principal curve so that it is suitable
# for ranking purposes, the anchor points p_0 and p_3 are modeled as a function of another parameter, alpha,
# as defined in Proposition 1.
with pm.Model() as model:
    point_sigma = pm.TruncatedNormal(
        name="point_sigma", 
        mu=point_sigma_prior, 
        sigma=sigma_prior_sigma, 
        lower=near_zero, 
        upper=point_sigma_prior + sigma_prior_upper_bound_delta - near_zero
        )
    score_sigma = pm.TruncatedNormal(
        name="score_sigma", 
        mu=score_sigma_prior,
        sigma=sigma_prior_sigma, 
        lower=near_zero, 
        upper=score_sigma_prior + sigma_prior_upper_bound_delta - near_zero
        )
    X_reconstruction_sigma = pm.TruncatedNormal(
        name="X_reconstruction_sigma", 
        mu=X_reconstruction_sigma_prior,
        sigma=sigma_prior_sigma,
        lower=near_zero, 
        upper=X_reconstruction_sigma_prior + sigma_prior_upper_bound_delta - near_zero
        )
    alpha = pm.Beta(
        name="alpha", 
        mu=alpha_prior, 
        sigma=point_sigma
        )
    p_0 = pm.Deterministic(
        name="p_0", var=.5 * (1. - alpha)
        )
    p_1 = pm.Beta(
        name="p_1", 
        mu=p_1_prior, 
        sigma=point_sigma
        )
    p_2 = pm.Beta(
        name="p_2", 
        mu=p_2_prior, 
        sigma=point_sigma
        )
    p_3 = pm.Deterministic(
        name="p_3", var=.5 * (1. + alpha)
        )
    s = pm.Beta(
        name="s", 
        mu=s_prior, 
        sigma=score_sigma
        )

    # Here, we define our likelihood function based on Formulas (15) and (23).
    Z = pm.math.stack(
        [pt.ones((num_scores,)),
         s,
         s ** 2.,
         s ** 3.],
         axis=0
    )
    M = pt.as_tensor(
        [[1., -3.,  3., -1.], 
         [0.,  3., -6.,  3.],
         [0.,  0.,  3., -3.], 
         [0.,  0.,  0.,  1.]]
        )
    P = pm.math.stack(
        [pt.transpose(p_0), 
         pt.transpose(p_1), 
         pt.transpose(p_2), 
         pt.transpose(p_3)], 
        axis=1
        )
    X_reconstruction_mu = pm.math.matmul(pm.math.matmul(P, M), Z)

    for i in range(dims):
        X_reconstruction = pm.Beta(
            name=f"X_reconstruction_{i}", 
            mu=X_reconstruction_mu[i, :], 
            sigma=X_reconstruction_sigma[i],
            observed=X[i, :]
            )

In [ ]:
# sanity check our model construction using a graphical visualization
pm.model_to_graphviz(model)

In [ ]:
# Let's visualize our prior predictive distribution to 1) confirm that our model is formulated as expected, and 2) sanity check
# the ability of he sampler to work with the bounds that we defined on our priors.
with model:
    predictions = pm.sample_prior_predictive(samples=1000, random_seed=rng)

az.plot_ppc(predictions, group="prior")

In [ ]:
# Here, we hit the inference button, making sure to initialize the algorithm at a point in the search space that we have previously
# demonstrated to yield a valid log probability calulation result.
initvals = {
    "point_sigma": point_sigma_prior,
    "score_sigma": score_sigma_prior,
    "X_reconstruction_sigma": X_reconstruction_sigma_prior,
    "alpha": alpha_prior,
    "p_1": p_1_prior,
    "p_2": p_2_prior,
    "s": s_prior
    }

# The pymc sampler has a C implementation that runs on CPU. One alternative sampler that runs on GPU is "numpyro." 
with model:
    idata1 = pm.sample(
        nuts_sampler="pymc",
        nuts_sampler_kwargs={"chain_method": "vectorized"},
        target_accept=.99,
        random_seed=rng,
        tune=1000,
        draws=1000,
        chains=4,
        cores=4,
        initvals=initvals
    )

In [ ]:
# Given a fit model, lets see how close our reconstructions are to our true data. We can see that the fit is good, but not perfect.
# We may get a better fit, and one that is more robust to different shapes of data distributions, by using mixtures of Beta 
# distributions for each feature dimension, where a Dirichlet prior on mixture weights would allow the optimal number of mixture 
# components to be learned for any given data set.
with model:
    idata1 = pm.sample_posterior_predictive(
        idata1, extend_inferencedata=True, random_seed=rng)

In [ ]:
# Here, and below, we visualize and save the posterior distribution over latent model vairables that we have learned
date_time = "2024-05-22_0947"
az.to_netcdf(idata1, f"./results/rpc_pymc_{date_time}.nc")

In [ ]:
ppc_plot = az.plot_ppc(idata1)
plt.savefig(f'./results/rpc_pymc_{date_time}_ppc_plot.png')

In [ ]:
alpha_trace_plot = az.plot_trace(idata1, var_names=['alpha'])
fig = alpha_trace_plot.flatten()[0].get_figure()
fig.savefig(f'./results/rpc_pymc_{date_time}_alpha_trace_plot.png')

In [ ]:
p_0_trace_plot = az.plot_trace(idata1, var_names=['p_0'])
fig = p_0_trace_plot.flatten()[0].get_figure()
fig.savefig(f'./results/rpc_pymc_{date_time}_p_0_trace_plot.png')

In [ ]:
p_1_trace_plot = az.plot_trace(idata1, var_names=['p_1'])
fig = p_1_trace_plot.flatten()[0].get_figure()
fig.savefig(f'./results/rpc_pymc_{date_time}_p_1_trace_plot.png')

In [ ]:
p_2_trace_plot = az.plot_trace(idata1, var_names=['p_2'])
fig = p_2_trace_plot.flatten()[0].get_figure()
fig.savefig(f'./results/rpc_pymc_{date_time}_p_2_trace_plot.png')

In [ ]:
p_3_trace_plot = az.plot_trace(idata1, var_names=['p_3'])
fig = p_3_trace_plot.flatten()[0].get_figure()
fig.savefig(f'./results/rpc_pymc_{date_time}_p_3_trace_plot.png')

In [ ]:
point_sigma_trace_plot = az.plot_trace(idata1, var_names=['point_sigma'])
fig = point_sigma_trace_plot.flatten()[0].get_figure()
fig.savefig(f'./results/rpc_pymc_{date_time}_point_sigma_trace_plot.png')

In [ ]:
score_sigma_trace_plot = az.plot_trace(idata1, var_names=['score_sigma'])
fig = score_sigma_trace_plot.flatten()[0].get_figure()
fig.savefig(f'./results/rpc_pymc_{date_time}_score_sigma_trace_plot.png')

In [ ]:
X_reconstruction_sigma_trace_plot = az.plot_trace(idata1, var_names=['X_reconstruction_sigma'])
fig = X_reconstruction_sigma_trace_plot.flatten()[0].get_figure()
fig.savefig(f'./results/rpc_pymc_{date_time}_X_reconstruction_sigma_trace_plot.png')

In [ ]:
# Here we visualize just one country's score distribution at a time. In the cell below, we visualize of all scores in a single plot
posterior = idata1.posterior.stack(sample=("chain", "draw"))
plt.hist(posterior["s"][0], 25, alpha=0.2, color='k')
plt.savefig(f'./results/rpc_pymc_{date_time}_s_hist_plot.png')

In [ ]:
posterior_s = idata1.posterior['s']
mean_score = posterior_s.mean(('chain', 'draw'))
y = np.linspace(0, 1, len(mean_score))
hdi = az.hdi(posterior_s).sortby(mean_score)
plt.plot(mean_score.sortby(mean_score), y)
plt.fill_betweenx(y, hdi['s'].values[:, 0], hdi['s'].values[:, 1], alpha=0.3)
plt.savefig(f'{date_time}_scores_hdi_plot.png')

In [ ]:
scores_mean = np.mean(posterior["s"], axis=1)
gapminder_series.insert(4, "score", scores_mean)
# gapminder_series["score"] = scores
gapminder_series

In [ ]:
# View the sorted scores (top and bottom ~5) to get a hint of how good the rankings are.
gapminder_series.sort_values("score", inplace=True, ascending=False)
gapminder_series

In [ ]:
# Write our data set out to disk with an added "score" column and sorted
gapminder_series.to_csv(f"./data/gapminder_quality_of_life_{date_time}_with_scores.csv")